### ODYM video, Stefan Pauliuk

Description of how MFA works according to the explaination of Pauliuk in the video. 


1. DynamicStockModel: is an instance requiring: years, stock, lifetime of the stock
2. for buildings, we have to go further back in time until we take stocks that arent currently being on use (?)
3. ripples in the graph come from changes in the stock flow from economic changes (this is represented in the database)
4. how much scrap is coming out from the MFA?  


### Flodym brightcon 2025 video, Jakob Dürrwächter
1. MFA tracks material thrpough system along life cycle stages
    1. it provides stock and flow time dynamics (future demands, temporal inter-dependency, circular economy interventions)

2. just one material a whole map throughout cycles
    3. there is a time delayed of the production of the product vs the end of life of the product as it stays in use for some years
3. we can describe the system environment by different dimensions: time, region, grade, product. But not all arrays have dimentions 
4. flodym makes alos plotting and data export so no need for different package
5. ODYM use to have many numpy operations outside the package, now FLODYM integrates them and make better data management. Also integratios pandas
6. There is REMIND-MFA with linking REMIND IAMs. from historical stocks to 2100 (demand of material in the future per country) crazyyyy

### Flodym course from brightcon 2025 

https://github.com/Depart-de-Sentier/brightcon-2025-material/blob/main/courses/advanced/flodym_course

In [ ]:
# 2_dimensions.ipynb 

import flodym as fd

#dimension is defined by name, letter and items

regions = fd.Dimension(name="Regions", letter="r", items=["EU", "US"])
print(regions)

# multiple dimensions are grouped in a DimensionSet object. Remember that dimension letters should be unique
time = fd.Dimension(name='Time', letter='t', items=[ 2020, 2023, 2025])
product = fd.Dimension(name='Product', letter='p', items=['Vehicles', 'Buildings'])
grade = fd.Dimension(name='Grade', letter='g', items=['Carbon Steel', 'Stainless'])

dims = fd.DimensionSet(dim_list=[time, regions, product, grade])
print(dims)

# DimensionSet indexing

print(dims['r']) # return regions
print(dims['Regions']) # return regions
print(dims['r', 'p'])
print(dims[('r',)]) #DimensionSet of just one item

Dimension 'Regions' ('r'); 2 items: ['EU', 'US']
DimensionSet (t,r,p,g) with shape (3, 2, 2, 2):
  't': 'Time' with length 3
  'r': 'Regions' with length 2
  'p': 'Product' with length 2
  'g': 'Grade' with length 2
Dimension 'Regions' ('r'); 2 items: ['EU', 'US']
Dimension 'Regions' ('r'); 2 items: ['EU', 'US']
DimensionSet (r,p) with shape (2, 2):
  'r': 'Regions' with length 2
  'p': 'Product' with length 2
DimensionSet (r) with shape (2,):
  'r': 'Regions' with length 2


In [ ]:
# 3_arrays.ypnb

# use same DimensionSet as notebook 2_dimensions.ipynb

# a FlodymArray is defined over a dimensionSet to make MFAs operations like addition, substraction or multiplication, division

production = fd.FlodymArray(dims=dims['t', 'r', 'g']) # Optional parameters: name (default "unnamed"), values (default all zeros)
print(production)

import numpy as np

values = 0.2 * np.ones(dims["t", "r", "g"].shape)
flow_production = fd.FlodymArray(name='production', dims=dims['t', 'r', 'g'], values=values)
print(flow_production)

# when making the operations, the results stored in the dimensions, neet to have an elipsis slice because flows are stored as a dictionary 
prm_loss_rate = fd.FlodymArray(dims=dims["t",], values=0.15 * np.ones(dims["t",].shape)) # in each time slice there is a 0.15 loss rate
flow_losses = fd.FlodymArray(dims=dims["r", "t"], values=0.04 * np.ones(dims["r", "t"].shape))
print(f"production dims: {str(flow_production.dims.letters)}; losses dims: {str(flow_losses.dims.letters)}")
# total loss 
print(flow_production + flow_losses) # adding or substraction, flodym would sum over g (or variable with extra dim), to make the dimensions match before adding. (If one array has extra dimensions, Flodym will sum over those extra dimensions before adding.) 
# loss rate per time step
print(flow_production * prm_loss_rate) # multiplication or division, the smaller array gets broadcasted (repeated) to match the longer array


FlodymArray 'unnamed' with dims (t,r,g) and shape (3, 2, 2);
Values:
[[[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]]]
FlodymArray 'production' with dims (t,r,g) and shape (3, 2, 2);
Values:
[[[0.2 0.2]
  [0.2 0.2]]

 [[0.2 0.2]
  [0.2 0.2]]

 [[0.2 0.2]
  [0.2 0.2]]]
production dims: ('t', 'r', 'g'); losses dims: ('r', 't')
FlodymArray 'unnamed' with dims (t,r) and shape (3, 2);
Values:
[[0.44 0.44]
 [0.44 0.44]
 [0.44 0.44]]
FlodymArray 'unnamed' with dims (t,r,g) and shape (3, 2, 2);
Values:
[[[0.03 0.03]
  [0.03 0.03]]

 [[0.03 0.03]
  [0.03 0.03]]

 [[0.03 0.03]
  [0.03 0.03]]]


In [ ]:
# control results in dimensions. flows are stored as dictionary
flow_losses[...] = flow_production * prm_loss_rate # This only works to reduce dimensions, throws an error if dimensions would have to be added. Always use elipsis slice
print(flow_losses)


FlodymArray 'unnamed' with dims (r,t) and shape (2, 3);
Values:
[[0.06 0.06 0.06]
 [0.06 0.06 0.06]]


In [ ]:
# 4_stocks.ipynb: stocks = accumulating materials

import numpy as np
import flodym as fd

dims = fd.DimensionSet(dim_list=[
    fd.Dimension(name="Time", letter="t", items=list(range(1970, 2060, 10))),
    fd.Dimension(name="Product", letter="p", items=["Vehicles", "Buildings"]),
])

# Inflow-driven means: You specify how much NEW material/products enter the system each year, and the model calculates:
# How the stock accumulates over time
# How much outflow (disposal/demolition) happens each year based on product lifetimes

my_dsm = fd.InflowDrivenDSM(
    dims=dims,
    lifetime_model=fd.NormalLifetime,
)
print(my_dsm)


InflowDrivenDSM 'unnamed' with dims (t,p) and shape (9, 2);
  Lifetime model: NormalLifetime


In [ ]:
lifetime_mean = fd.Parameter(dims=dims[("p",)], values=np.array([15, 60]))
my_dsm.inflow.values[...] = 1.
my_dsm.lifetime_model.set_prms(mean=lifetime_mean, std=0.3 * lifetime_mean)
my_dsm.compute()

print("Stock values for Buildings:") # The stock is the total number of buildings still in use at each point
print(my_dsm.stock["Buildings"].to_df())

Stock values for Buildings:
          value
Time           
1970   9.988768
1980  19.926671
1990  29.667462
2000  38.843129
2010  46.819845
2020  52.913930
2030  56.819845
2040  58.843129
2050  59.667462


In [ ]:
# 5_mfa_system.ipynb
# FlodymArray and StockArray objects work without MFASystem

# Three ingredients for MFA System
#   System Definition
#       Which dimensions and processes are there?
#       Which flows, parameters, stocks are there?
#       Which dimensions do they have?
#   Data input from files
#       dimension items (which years, regions, ...)
#       parameter values
#   Compute routine




In [1]:
# exercise_2ABC_solution.ipynb

# Definitions
import flodym as fd

dimension_definitions = [
    fd.DimensionDefinition(letter="t", name="time", dtype=int),
    fd.DimensionDefinition(letter="p", name="product", dtype=str),
    fd.DimensionDefinition(letter="e", name="element", dtype=str),
]

parameter_definitions = [
    fd.ParameterDefinition(name="production", dim_letters=("t",)),
    fd.ParameterDefinition(name="manufacturing loss rate", dim_letters=("t",)),
    fd.ParameterDefinition(name="product shares", dim_letters=("p",)),
    fd.ParameterDefinition(name="element shares", dim_letters=("e",)),
    fd.ParameterDefinition(name="product lifetimes", dim_letters=("p",)),
]

process_names = [
    "sysenv",
    "manufacturing",
    "use",
    "waste",
]

flow_definitions = [
    fd.FlowDefinition(from_process_name="sysenv", to_process_name="manufacturing", dim_letters=("t", "e")),
    fd.FlowDefinition(from_process_name="manufacturing", to_process_name="sysenv", dim_letters=("t", "e")),
    fd.FlowDefinition(from_process_name="manufacturing", to_process_name="use", dim_letters=("t", "p", "e")),
    fd.FlowDefinition(from_process_name="use", to_process_name="waste", dim_letters=("t", "p", "e")),
]

stock_definitions = [
    fd.StockDefinition(
        name="use",
        process="use",
        dim_letters=("t", "p", "e"),
        subclass=fd.InflowDrivenDSM,
        lifetime_model_class=fd.LogNormalLifetime,
    ),
        fd.StockDefinition(
        name="waste",
        process="waste",
        dim_letters=("t", "e"),
        subclass=fd.SimpleFlowDrivenStock,
    ),
]

mfa_definition = fd.MFADefinition(
    dimensions=dimension_definitions,
    parameters=parameter_definitions,
    processes=process_names,
    flows=flow_definitions,
    stocks=stock_definitions,
)

# Data sources
dimension_files = {
    "time": "data/dimension_time.csv",
    "product": "data/dimension_product.csv",
    "element": "data/dimension_element.csv",
}

parameter_files = {
    "production": "data/parameter_production.csv",
    "manufacturing loss rate": "data/parameter_manufacturing_loss_rate.csv",
    "product shares": "data/parameter_product_shares.csv",
    "element shares": "data/parameter_element_shares.csv",
    "product lifetimes": "data/parameter_product_lifetimes.csv",
}

# Compute routine

class SimpleMFA(fd.MFASystem):
    def compute(self):

        # manufacturing flows
        self.flows["sysenv => manufacturing"][...] = self.parameters["production"] * self.parameters["element shares"]
        self.flows["manufacturing => sysenv"][...] = self.flows["sysenv => manufacturing"] * self.parameters["manufacturing loss rate"]
        total_products = self.flows["sysenv => manufacturing"] - self.flows["manufacturing => sysenv"]
        self.flows["manufacturing => use"][...] = total_products * self.parameters["product shares"]

        # use stock
        self.stocks["use"].inflow[...] = self.flows["manufacturing => use"]
        self.stocks["use"].lifetime_model.set_prms(
            mean=self.parameters["product lifetimes"],
            std=0.5*self.parameters["product lifetimes"],
        )
        self.stocks["use"].compute()

        # end-of-life  flow
        self.flows["use => waste"][...] = self.stocks["use"].outflow

        # waste stock
        self.stocks["waste"].inflow[...] = self.flows["use => waste"][...]
        self.stocks["waste"].compute()

In [2]:
# Init, load, compute
mfa_example = SimpleMFA.from_csv(
    definition=mfa_definition,
    dimension_files=dimension_files,
    parameter_files=parameter_files,
)

mfa_example.compute()

In [3]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

mfa_example.check_mass_balance()

INFO:root:Checking mass balance of SimpleMFA object...
INFO:root:Success - Mass balance is consistent!


In [13]:
# Sankey plotting

import flodym.export as fde
import plotly.io as pio

# Force Plotly to use a browser-based renderer instead of notebook
pio.renderers.default = "browser"  # Opens in browser instead of inline

fig = fde.PlotlySankeyPlotter(mfa=mfa_example, exclude_processes=[]).plot()
fig.show()  # This will open in your web browser


# Exercise 2C
mfa_example.stocks["use"].stock.to_df().head()

value
time product   element          
1980 Vehicles  Fe       0.742526
               Cu       0.015154
     Buildings Fe       0.495018
               Cu       0.010102
1981 Vehicles  Fe       1.509152